In [183]:
import numpy as np
import random
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

In [196]:
df = pd.read_csv('train.csv')
df.head()

,Unnamed: 0,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,551,15806307,S2336,720,S2,Male,38,5,114051.97,2,0,1,107577.29,0
1,1,6897,15709621,S1500,682,S0,Female,54,4,62397.41,1,1,0,113088.60,1
2,2,4588,15619340,S1865,672,S0,Female,31,5,119903.67,1,1,1,132925.17,0
3,3,291,15620746,S1672,592,S2,Female,40,4,104257.86,1,1,0,110857.33,0
4,4,1673,15646372,S2532,753,S2,Male,42,5,120387.73,1,0,1,126378.57,0


In [228]:
CreditScore = df['CreditScore'].tolist()
Geography = df['Geography'].tolist()
Gender = df['Gender'].tolist()
Age = df['Age'].tolist()
Tenure = df['Tenure'].tolist()
Balance = df['Balance'].tolist()
NumOfProducts = df['NumOfProducts'].tolist()
HasCrCard = df['HasCrCard'].tolist()
IsActiveMember = df['IsActiveMember'].tolist()
EstimatedSalary = df['EstimatedSalary'].tolist()
Exited = df['Exited'].tolist()

mini = min(CreditScore)
maxi = max(CreditScore)-mini
for i, d in enumerate(CreditScore):
    CreditScore[i] = (CreditScore[i]-mini)/maxi
    
mini = min(Age)
maxi = max(Age)-mini
for i, d in enumerate(Age):
    Age[i] = (Age[i]-mini)/maxi
    
mini = min(Tenure)
maxi = max(Tenure)-mini
for i, d in enumerate(Tenure):
    Tenure[i] = (Tenure[i]-mini)/maxi
    
mini = min(Balance)
maxi = max(Balance)-mini
for i, d in enumerate(Balance):
    Balance[i] = (Balance[i]-mini)/maxi
    
mini = min(NumOfProducts)
maxi = max(NumOfProducts)-mini
for i, d in enumerate(NumOfProducts):
    NumOfProducts[i] = (NumOfProducts[i]-mini)/maxi
    
mini = min(HasCrCard)
maxi = max(HasCrCard)-mini
for i, d in enumerate(HasCrCard):
    HasCrCard[i] = (HasCrCard[i]-mini)/maxi
    
mini = min(IsActiveMember)
maxi = max(IsActiveMember)-mini
for i, d in enumerate(IsActiveMember):
    IsActiveMember[i] = (IsActiveMember[i]-mini)/maxi
    
mini = min(EstimatedSalary)
maxi = max(EstimatedSalary)-mini
for i, d in enumerate(EstimatedSalary):
    EstimatedSalary[i] = (EstimatedSalary[i]-mini)/maxi
    

In [230]:
rand_index = random.sample(set(range(len(Exited))), round(len(Exited)*0.8))
_df = df.drop(rand_index, inplace=False)
# for cnt, _ind in enumerate(rand_index, 1):
#     _df.loc[cnt] = df[_ind]
# _df = df[rand_index]
# _df.reset_index(drop=True, inplace=True)

# sns.relplot(x='CreditScore', y='Age', hue='Exited', data=_df)
# sns.relplot(x='Tenure', y='Balance', hue='Exited', data=_df)


In [234]:
train_indices = random.sample(set(range(len(Exited))), round(len(Exited)*0.8))
test_indices = list(set(range(len(Exited))) - set(train_indices))
train_y = []
train_x = []
for _ind in train_indices:
    train_x.append((CreditScore[_ind], Age[_ind], Tenure[_ind], Balance[_ind],\
                   NumOfProducts[_ind], HasCrCard[_ind], IsActiveMember[_ind], EstimatedSalary[_ind]))
    if Exited[_ind]==1:
        train_y.append(1)
    else:
        train_y.append(-1)

In [278]:
sess = tf.Session()

num_x = 8
batch_size = 128
lr_rate = 0.0001
gamma_val = -25.0
num_epoch = 64

x_data = tf.placeholder(shape=[None, num_x], dtype=tf.float32)
y_target = tf.placeholder(shape=[None, 1], dtype=tf.float32)
prediction_grid = tf.placeholder(shape=[None, num_x], dtype=tf.float32)
b = tf.Variable(tf.random_normal(shape=[1,batch_size]))

In [279]:
gamma = tf.constant(gamma_val)
dist = tf.reduce_sum(tf.square(x_data), 1)
dist = tf.reshape(dist, [-1,1])
sq_dists = tf.add(tf.subtract(dist, tf.multiply(float(num_x), tf.matmul(x_data,tf.transpose(x_data)))), tf.transpose(dist))
my_kernel = tf.exp(tf.multiply(gamma, tf.abs(sq_dists)))

model_output = tf.matmul(b, my_kernel)
first_term = tf.reduce_sum(b)
b_vec_cross = tf.matmul(tf.transpose(b), b)
y_target_cross = tf.matmul(y_target, tf.transpose(y_target))
second_term = tf.reduce_sum(tf.multiply(my_kernel, tf.multiply(b_vec_cross,y_target_cross)))
loss = tf.negative(tf.subtract(first_term, second_term))

rA = tf.reshape(tf.reduce_sum(tf.square(x_data), 1),[-1,1])
rB = tf.reshape(tf.reduce_sum(tf.square(prediction_grid), 1),[-1,1])
pred_sq_dist = tf.add(tf.subtract(rA, tf.multiply(float(num_x), tf.matmul(x_data,
tf.transpose(prediction_grid)))), tf.transpose(rB))
pred_kernel = tf.exp(tf.multiply(gamma, tf.abs(pred_sq_dist)))
prediction_output = tf.matmul(tf.multiply(tf.transpose(y_target),b),pred_kernel)
prediction = tf.sign(prediction_output-tf.reduce_mean(prediction_output))
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.squeeze(prediction),tf.squeeze(y_target)), tf.float32))

my_opt = tf.train.GradientDescentOptimizer(lr_rate)
train_step = my_opt.minimize(loss)
init = tf.initialize_all_variables()
sess.run(init)

In [280]:
loss_vec = []
batch_accuracy = []
x_vals = np.array(train_x)
y_vals = np.array(train_y)

rand_index = list(range(len(x_vals)))
pt = 0
while(num_epoch):
    if pt+batch_size >= len(x_vals):
        random.shuffle(rand_index)
        num_epoch-=1
        pt = 0
    X = x_vals[rand_index[pt:pt+batch_size]]
    Y = np.transpose([y_vals[rand_index[pt:pt+batch_size]]])
    pt+=batch_size
    sess.run(train_step, feed_dict={x_data: X, y_target:Y})
    temp_loss = sess.run(loss, feed_dict={x_data: X, y_target: Y})
    loss_vec.append(temp_loss)
    acc_temp = sess.run(accuracy, feed_dict={x_data: X,y_target: Y,prediction_grid:X})
    batch_accuracy.append(acc_temp)


In [281]:
batch_accuracy

[0.7109375,
 0.75,
 0.6953125,
 0.7265625,
 0.6953125,
 0.7265625,
 0.734375,
 0.265625,
 0.328125,
 0.765625,
 0.2734375,
 0.7734375,
 0.6484375,
 0.703125,
 0.28125,
 0.71875,
 0.671875,
 0.734375,
 0.3671875,
 0.671875,
 0.2734375,
 0.6953125,
 0.765625,
 0.671875,
 0.71875,
 0.7109375,
 0.7265625,
 0.6875,
 0.6640625,
 0.765625,
 0.1796875,
 0.3046875,
 0.40625,
 0.3203125,
 0.7109375,
 0.734375,
 0.65625,
 0.7421875,
 0.203125,
 0.6796875,
 0.3359375,
 0.75,
 0.296875,
 0.6875,
 0.265625,
 0.8203125,
 0.7578125,
 0.1953125,
 0.734375,
 0.7109375,
 0.7265625,
 0.6171875,
 0.734375,
 0.203125,
 0.2734375,
 0.3046875,
 0.703125,
 0.71875,
 0.6640625,
 0.7109375,
 0.703125,
 0.6875,
 0.2421875,
 0.3125,
 0.25,
 0.671875,
 0.6796875,
 0.28125,
 0.7109375,
 0.25,
 0.734375,
 0.3125,
 0.6796875,
 0.7421875,
 0.828125,
 0.28125,
 0.7421875,
 0.7578125,
 0.75,
 0.34375,
 0.75,
 0.2578125,
 0.3671875,
 0.6796875,
 0.71875,
 0.6640625,
 0.703125,
 0.7421875,
 0.7421875,
 0.3046875,
 0.640625

In [277]:
loss_vec

[-39.084145,
 -20.829582,
 -18.963036,
 -25.174646,
 -27.505753,
 -31.354069,
 -29.224634,
 -20.57982,
 -29.437302,
 -51.721237,
 -19.905853,
 -36.248035,
 -30.614239,
 -19.92913,
 -25.184502,
 -20.21233,
 -23.459822,
 -8.818762,
 -28.477917,
 -20.583115,
 -36.22243,
 -33.063892,
 -22.503918,
 -27.804682,
 -24.12378,
 -2.5291386,
 -23.370455,
 -30.79415,
 -27.010029,
 -32.594975,
 -7.522373,
 -23.66408,
 -30.52029,
 -34.50117,
 -23.394205,
 -28.172909,
 -27.941986,
 -39.74671,
 -44.99667,
 -41.646107,
 -48.79592,
 -21.446095,
 -43.92756,
 -23.146778,
 -31.322094,
 -32.049343,
 -36.261795,
 -42.581787,
 -30.348612,
 -33.44523,
 -37.73568,
 -34.67502,
 -35.26782,
 -25.857967,
 -38.795467,
 -26.499008,
 -41.296722,
 -26.855532,
 -36.80116,
 -14.4633045,
 -40.571243,
 -48.339256,
 -45.884594,
 -49.780964,
 -40.23096,
 -36.73263,
 -37.511177,
 -35.464676,
 -36.208637,
 -47.636013,
 -30.939417,
 -44.728176,
 -50.26023,
 -35.60041,
 -28.07955,
 -52.369255,
 -39.301308,
 -28.004326,
 -40.66510